In [1]:
# standard libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os,warnings
warnings.simplefilter('ignore')
from sklearn.model_selection import train_test_split
from tabulate import tabulate
import datetime
Table = []

In [2]:
TRAIN_PATH = r"C:\Users\marnt\Desktop\kaggle\titanic\train.csv"
TEST_PATH = r"C:\Users\marnt\Desktop\kaggle\titanic\test.csv"
SUBMISSION_PATH = r"C:\Users\marnt\Desktop\kaggle\titanic\submit.csv"
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)
submit_df = pd.read_csv(SUBMISSION_PATH)

In [3]:
print('Missing values Percentage: \n\n', round (train_df.isnull().sum().sort_values(ascending=False)/len(train_df)*100,1))

Missing values Percentage: 

 Cabin          77.1
Age            19.9
Embarked        0.2
PassengerId     0.0
Name            0.0
Pclass          0.0
Survived        0.0
Sex             0.0
Parch           0.0
SibSp           0.0
Fare            0.0
Ticket          0.0
dtype: float64


In [4]:
#train_df['Embarked'] = train_df['Embarked'].fillna(train_df['Embarked'].mode())
#train_df['Embarked'].replace({'C':0, 'Q':1, 'S':2}, inplace = True)
import statistics
embarked_mode = statistics.mode(train_df['Embarked'])
for i in range(len(train_df['Embarked'])):
    if type(train_df['Embarked'][i]) != str:
        train_df['Embarked'][i] = embarked_mode


##More sophisticated method for this would look into fares paid at each port, but for two values i don't think it's very impactful on the results

In [5]:
#This is a way of imputing missing values for the fares in the test data, but it's also a way to make use of the data present in the ticket field that is otherwise difficult to use for this task
PeopleInTicket = []
for i in range(len(train_df['Ticket'])):
    PeopleInTicket.append(train_df['Ticket'].value_counts()[train_df['Ticket'][i]])
train_df['PeopleInTicket'] = PeopleInTicket

In [6]:
#Imputing missing values for the age field, this is done by looking at peoples title (i.e. Mr, Ms) and separating by class
title_lst= []
for i in range(len(train_df['Name'])):
    x = train_df['Name'][i].split(',')[1]
    title = x.split('.')[0]
    title = title.lstrip()
    title_lst.append(title)
train_df['Title'] = title_lst

title_dict = {'Capt':'Military', 'Col':'Military', 'Don':'Royalty', 'Dr':'Mr_Special', 'Jonkheer':'Upper', 'Lady':'Mrs', 'Major':'Military', 'Master':'Master',
              'Miss':'Ms', 'Mlle':'Ms', 'Mme':'Mrs', 'Mr':'Mr', 'Mrs':'Mrs', 'Ms':'Ms', 'Rev':'Mr_Special', 'Sir':'Upper', 'the Countess':'Upper', 'Dona':'Upper'}
for i in range(len(train_df['Title'])):
    train_df['Title'][i] = title_dict[train_df['Title'][i]]

#age_mean_byTitle = train_df.groupby(['Title', 'Pclass'])['Age'].agg(['mean', 'count'])
#print(age_mean_byTitle)

for i in range(len(train_df['Age'])):
    if np.isnan(train_df['Age'][i]):
        if train_df['Title'][i] == 'Ms': #There are significant differences in the Ms title depending on class, can be seen in the above commented out command                              
            if train_df['Parch'][i] > 0:
                train_df['Age'][i] = (train_df[(train_df.Title=="Ms") & (train_df.Parch!=0)]['Age'].mean())
            else:
                train_df['Age'][i] = (train_df[(train_df.Title=="Ms") & (train_df.Parch==0)]['Age'].mean())
        else:
            train_df['Age'][i] = (train_df[(train_df.Title == train_df['Title'][i]) & (train_df.Pclass == train_df['Pclass'][i])]['Age'].mean())

In [7]:
print('Missing values Percentage: \n\n', round (test_df.isnull().sum().sort_values(ascending=False)/len(test_df)*100,1))

Missing values Percentage: 

 Cabin          78.2
Age            20.6
Fare            0.2
Name            0.0
Pclass          0.0
PassengerId     0.0
Sex             0.0
Parch           0.0
SibSp           0.0
Ticket          0.0
Embarked        0.0
dtype: float64


In [8]:
#Imputing the same missing values in the test data as done above for the training data, this time including imputing the missing fare values
PeopleInTicket = []
for i in range(len(test_df['Ticket'])):
    PeopleInTicket.append(test_df['Ticket'].value_counts()[test_df['Ticket'][i]])
test_df['PeopleInTicket'] = PeopleInTicket

test_df['FarePerPerson']=test_df['Fare']/test_df['PeopleInTicket']

for i in range(len(test_df['Fare'])):
    if np.isnan(test_df['Fare'][i]):
        if test_df['Embarked'][i] == 'C':
            fare = test_df[(test_df.Embarked=='C') & (test_df.Pclass == test_df['Pclass'][i])]['Fare'].mean()
            test_df['Fare'][i] = fare*test_df['PeopleInTicket'][i]
        elif test_df['Embarked'][i] == 'S':
            fare = test_df[(test_df.Embarked=='S') & (test_df.Pclass == test_df['Pclass'][i])]['Fare'].mean()
            test_df['Fare'][i] = fare*test_df['PeopleInTicket'][i]
        else:
            fare = test_df[(test_df.Embarked=='Q') & (test_df.Pclass == test_df['Pclass'][i])]['Fare'].mean()
            test_df['Fare'][i] = fare*test_df['PeopleInTicket'][i]
#fares_embarkment = test_df.groupby(['Embarked', 'Pclass'])['FarePerPerson'].agg(['mean', 'count'])
#print(fares_embarkment)

In [9]:
title_lst= []
for i in range(len(test_df['Name'])):
    x = test_df['Name'][i].split(',')[1]
    title = x.split('.')[0]
    title = title.lstrip()
    title_lst.append(title)
test_df['Title'] = title_lst

title_dict = {'Capt':'Military', 'Col':'Military', 'Don':'Royalty', 'Dr':'Mr_Special', 'Jonkheer':'Upper', 'Lady':'Mrs', 'Major':'Military', 'Master':'Master',
              'Miss':'Ms', 'Mlle':'Ms', 'Mme':'Mrs', 'Mr':'Mr', 'Mrs':'Mrs', 'Ms':'Ms', 'Rev':'Mr_Special', 'Sir':'Upper', 'the Countess':'Upper', 'Dona':'Upper'}
for i in range(len(test_df['Title'])):
    test_df['Title'][i] = title_dict[test_df['Title'][i]]

for i in range(len(test_df['Age'])):
    if np.isnan(test_df['Age'][i]):
        if test_df['Title'][i] == 'Ms':
            if test_df['Parch'][i] > 0:
                test_df['Age'][i] = (test_df[(test_df.Title=="Ms") & (test_df.Parch!=0)]['Age'].mean())
            else:
                test_df['Age'][i] = (test_df[(test_df.Title=="Ms") & (test_df.Parch==0)]['Age'].mean())
        else:
            test_df['Age'][i] = (test_df[(train_df.Title == test_df['Title'][i]) & (test_df.Pclass == test_df['Pclass'][i])]['Age'].mean())

In [10]:
#Separating out predictor and outcome variables
Y = train_df['Survived']

train_df['Sex'].replace({'male':1, 'female':0}, inplace= True)
train_df['Embarked'].replace({'C':0, 'Q':1, 'S':2}, inplace = True)
train_df['Title'].replace({'Mr':0, 'Ms':1, 'Mrs':2, 'Master':3, 'Mr_Special':4, 'Military':5, 'Upper':6, 'Royalty':7}, inplace = True)
X = train_df.drop(['Survived', 'Name', 'Ticket', 'Cabin'], axis = 1)
#X = X.drop(['PassengerId', 'PeopleInTicket', 'Title'], axis=1)

print(X)


test_df['Sex'].replace({'male':1, 'female':0}, inplace = True)
test_df['Embarked'].replace({'C':0, 'Q':1, 'S':2}, inplace = True)
test_df['Title'].replace({'Mr':0, 'Ms':1, 'Mrs':2, 'Master':3, 'Mr_Special':4, 'Military':5, 'Upper':6, 'Royalty':7}, inplace = True)
test_df = test_df.drop(['Name', 'Cabin', 'Ticket', 'FarePerPerson'], axis = 1)
#test_df = test_df.drop(['PassengerId', PeopleInTicket', 'Title'])
print(test_df)


     PassengerId  Pclass  Sex        Age  SibSp  Parch     Fare  Embarked  \
0              1       3    1  22.000000      1      0   7.2500         2   
1              2       1    0  38.000000      1      0  71.2833         0   
2              3       3    0  26.000000      0      0   7.9250         2   
3              4       1    0  35.000000      1      0  53.1000         2   
4              5       3    1  35.000000      0      0   8.0500         2   
..           ...     ...  ...        ...    ...    ...      ...       ...   
886          887       2    1  27.000000      0      0  13.0000         2   
887          888       1    0  19.000000      0      0  30.0000         2   
888          889       3    0  11.863636      1      2  23.4500         2   
889          890       1    1  26.000000      0      0  30.0000         0   
890          891       3    1  32.000000      0      0   7.7500         1   

     PeopleInTicket  Title  
0                 1      0  
1                

In [11]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0)

In [12]:
#Multinomial Naive Bayes Classifier
# import the library
from sklearn.naive_bayes import MultinomialNB

# instantiate & fit
mnb = MultinomialNB().fit(x_train, y_train)
print("score on test: " + str(mnb.score(x_test, y_test)))
Table.append(['Naive Bayes', mnb.score(x_test, y_test)])

score on test: 0.6816143497757847


In [13]:
#Logistic Regression Classifier
# import the library
from sklearn.linear_model import LogisticRegression

# instantiate & fit
lr=LogisticRegression(max_iter=5000)
lr.fit(x_train, y_train)
print("score on test: " + str(lr.score(x_test, y_test)))
Table.append(['Logistic Regression', lr.score(x_test, y_test)])

score on test: 0.7847533632286996


In [14]:
#Stochastic Gradient Descent Classifier
# import the library
from sklearn.linear_model import SGDClassifier

# instantiate & fit
sgd=SGDClassifier()
sgd.fit(x_train, y_train)
print("score on test: " + str(sgd.score(x_test, y_test)))
Table.append(['SGDClassifier', sgd.score(x_test, y_test)])

score on test: 0.6412556053811659


In [15]:
#K-Nearest Neighbours Classifier
# import the library
from sklearn.neighbors import KNeighborsClassifier

# instantiate & fit
knn = KNeighborsClassifier(algorithm = 'brute', n_jobs=-1)
knn.fit(x_train, y_train)
print("score on test: " + str(knn.score(x_test, y_test)))
Table.append(['KNN', knn.score(x_test, y_test)])

score on test: 0.6053811659192825


In [16]:
#Linear Support Vector Classifier
# import the library
from sklearn.svm import LinearSVC

# instantiate & fit
svm=LinearSVC(C=0.0001)
svm.fit(x_train, y_train)
print("score on test: " + str(svm.score(x_test, y_test)))
Table.append(['SVM', svm.score(x_test, y_test)])

score on test: 0.7085201793721974


In [17]:
#Decision Tree Classifier
# import the library
from sklearn.tree import DecisionTreeClassifier

# instantiate & fit
clf = DecisionTreeClassifier(min_samples_split=10,max_depth=3)
clf.fit(x_train, y_train)
print("score on test: "  + str(clf.score(x_test, y_test)))
Table.append(['Decision Tree', clf.score(x_test, y_test)])

score on test: 0.7937219730941704


In [18]:
#Bagging Classifier
# import the library
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# instantiate & fit
bg=BaggingClassifier(DecisionTreeClassifier(min_samples_split=10,max_depth=3),max_samples=0.5,max_features=1.0,n_estimators=10)
bg.fit(x_train, y_train)
print("score on test: " + str(bg.score(x_test, y_test)))
Table.append(['Bagging Decision Tree', bg.score(x_test, y_test)])

score on test: 0.7937219730941704


In [19]:
#Adaptive Boosting Classifier
# import the library
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# instantiate & fit
adb = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2),n_estimators=100,learning_rate=0.5)
adb.fit(x_train, y_train)
print("score on test: " + str(adb.score(x_test, y_test)))
Table.append(['AdaBoost Classifier', adb.score(x_test, y_test)])

score on test: 0.8071748878923767


In [20]:
#Gradient Boosting Classifier
# import the library
from sklearn.ensemble import GradientBoostingClassifier

# instantiate & fit
gbc = GradientBoostingClassifier(n_estimators=100)
gbc.fit(x_train, y_train)
print("score on test: " + str(gbc.score(x_test, y_test)))
Table.append(['Gradient Boost Classifier', gbc.score(x_test, y_test)])

score on test: 0.8116591928251121


In [21]:
#Random Forest Classifier
# import the library
from sklearn.ensemble import RandomForestClassifier

# instantiate & fit
rf = RandomForestClassifier(n_estimators=300,max_depth=3)
rf.fit(x_train, y_train)
print("score on test: " + str(rf.score(x_test, y_test)))
Table.append(['Random Forest', rf.score(x_test, y_test)])

score on test: 0.8071748878923767


In [22]:
#Voting Classifier
# import the library
from sklearn.ensemble import VotingClassifier

# 1) naive bias = mnb
mnb = MultinomialNB().fit(x_train, y_train)
# 2) logistic regression =lr
lr=LogisticRegression(max_iter=5000)
# 3) random forest =rf
rf = RandomForestClassifier(n_estimators=30,max_depth=3)
# 4) suport vecotr mnachine = svm
svm=LinearSVC(max_iter=5000)
evc=VotingClassifier(estimators=[('mnb',mnb),('lr',lr),('rf',rf),('svm',svm)])
evc.fit(x_train, y_train)

print("score on test: " + str(evc.score(x_test, y_test)))
Table.append(['Voting Classifier', evc.score(x_test, y_test)])

score on test: 0.7847533632286996


In [23]:
print(tabulate(Table, headers=["Model","Score"], tablefmt='fancy_outline') )

╒═══════════════════════════╤══════════╕
│ Model                     │    Score │
╞═══════════════════════════╪══════════╡
│ Naive Bayes               │ 0.681614 │
│ Logistic Regression       │ 0.784753 │
│ SGDClassifier             │ 0.641256 │
│ KNN                       │ 0.605381 │
│ SVM                       │ 0.70852  │
│ Decision Tree             │ 0.793722 │
│ Bagging Decision Tree     │ 0.793722 │
│ AdaBoost Classifier       │ 0.807175 │
│ Gradient Boost Classifier │ 0.811659 │
│ Random Forest             │ 0.807175 │
│ Voting Classifier         │ 0.784753 │
╘═══════════════════════════╧══════════╛


In [24]:
# fine-tuning the Gradient Boost Classifier
from sklearn.model_selection import GridSearchCV

# instantiate
gbcc = GradientBoostingClassifier(n_estimators=100)

# define paramter grid.
param_grid = [{'min_samples_split': [5, 10, 15, 20], 'max_depth': [3, 6, 9, 12]}]

# define grid-search
grid_search = GridSearchCV(estimator=gbcc,
                          param_grid=param_grid,
                          scoring="top_k_accuracy",
                          cv=5,
                          return_train_score=True)

# fit the grid search
grid_search.fit(x_train, y_train)

,estimator,GradientBoostingClassifier()
,param_grid,"[{'max_depth': [3, 6, ...], 'min_samples_split': [5, 10, ...]}]"
,scoring,'top_k_accuracy'
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,True
,loss,'log_loss'


In [25]:
# get the best estimator
gbcc_tuned = grid_search.best_estimator_

# fit the estimator
gbcc_tuned.fit(x_train, y_train)
print("score on test: "  + str(gbcc_tuned.score(x_test, y_test)))

score on test: 0.820627802690583


In [26]:
# make prediction on TEST data
result = gbcc_tuned.predict(test_df)

# add the prediction to the submission
submit_df['Survived'] = result

# get the datestamp
datestamp = '{:%Y_%m_%d}'.format(datetime.date.today())

# save the submission
submit_df.to_csv(str(datestamp)+'_GradientBoost_'+"_submission.csv",index=False)

In [27]:
#Quantifying importance of predictor variables
importances = gbcc_tuned.feature_importances_
sorted_features = sorted(zip(test_df.columns, importances), key=lambda x: -x[1])

print("Feature importances:")
for feature, score in sorted_features:
    print(f"{feature}: {score:.4f}")

#Title > Fare > PeopleInTicket > PassengerID(?) > Age > Pclass > Embarked > SibSp > Sex > Parch     
#Only title and fare have a score > 0.1, sex and parch are both < 0.01

Feature importances:
Title: 0.4940
Fare: 0.1172
PeopleInTicket: 0.0917
PassengerId: 0.0894
Age: 0.0869
Pclass: 0.0725
Embarked: 0.0207
SibSp: 0.0200
Sex: 0.0053
Parch: 0.0024


In [29]:
#Trying to find the best subset of predictor variables to use

from mlxtend.feature_selection import ExhaustiveFeatureSelector

efs = ExhaustiveFeatureSelector(
    estimator=gbcc_tuned,
    min_features=1,
    max_features=test_df.shape[1],
    scoring="accuracy",
    cv=5
)

efs = efs.fit(x_train, y_train)

print("Best subset:", efs.best_feature_names_)
print("Best CV score:", efs.best_score_)

Features: 1023/1023

Best subset: ('Pclass', 'Age', 'Fare', 'Embarked', 'PeopleInTicket', 'Title')
Best CV score: 0.8443272360004489


In [39]:
#X.columns // PassengerID, Pclass, Sex, Age, SibSp, Parch, Fare, Embarked, PeopleInTicket, Title
#Best subset: Pclass, Age, Fare, Embarked, PeopleInTicket, Title
#NOTE: The predictions formed from these predictors solely perform slightly worse on the test data
X_subset = X[list(efs.best_feature_names_)]
test_df_subset = test_df[list(efs.best_feature_names_)]

x_train_subset, x_test_subset, y_train_subset, y_test_subset = train_test_split(X_subset, Y, random_state=0)

# import the library
from sklearn.ensemble import GradientBoostingClassifier

# instantiate & fit
gbc = GradientBoostingClassifier(n_estimators=1000)
gbc.fit(x_train_subset, y_train_subset)
print("score on test: " + str(gbc.score(x_test_subset, y_test_subset)))


# fine-tuning the Gradient Boosting Classifier
from sklearn.model_selection import GridSearchCV

# instantiate
gbc_subset = GradientBoostingClassifier(n_estimators=100)

# define paramter grid.
param_grid = [{'min_samples_split': [5, 10, 15, 20, 25, 30], 'max_depth': [3, 6, 9, 12, 15, 18]}]

# define grid-search
grid_search = GridSearchCV(estimator=gbc_subset,
                          param_grid=param_grid,
                          scoring="top_k_accuracy",
                          cv=20,
                          return_train_score=True)

# fit the grid search
grid_search.fit(x_train_subset, y_train_subset)

# get the best estimator
gbc_tuned_subset = grid_search.best_estimator_

# fit the estimator
gbc_tuned_subset.fit(x_train_subset, y_train_subset)
print("score on test: "  + str(gbcc_tuned.score(x_test_subset, y_test_subset)))

# make prediction on TEST data
result = gbc_tuned_subset.predict(test_df_subset)

# add the prediction to the submission
submit_df['Survived'] = result

# get the datestamp
datestamp = '{:%Y_%m_%d}'.format(datetime.date.today())

# save the submission
submit_df.to_csv(str(datestamp)+'_GradientBoost_'+'_SubsetOfPredictors_'+"_submission.csv",index=False)

score on test: 0.8071748878923767
score on test: 0.8161434977578476
